* notebook created by nov05 on 2024-12-01  
* local conda env [`awsmle_py310`](https://gist.github.com/nov05/d9c3be6c2ab9f6c050e3d988830db08b) (no cuda)     
* AWS SageMaker script mode  

---   

* https://sagemaker.readthedocs.io/en/v2.34.0/frameworks/pytorch/sagemaker.pytorch.html   
* https://docs.wandb.ai/guides/integrations/sagemaker/  

In [ ]:
# TODO: Install any packages that you might need
# !pip install smdebug

In [8]:
!notepad C:\Users\guido\.aws\credentials

In [5]:
## reset the session after updating credentials
import boto3 # type: ignore
boto3.DEFAULT_SESSION = None

import sagemaker # type: ignore
from sagemaker import get_execution_role # type: ignore
role_arn = get_execution_role()  ## get role ARN
if 'AmazonSageMaker-ExecutionRole' not in role_arn:
    ## your own role here
    role_arn = "arn:aws:iam::061096721307:role/service-role/AmazonSageMaker-ExecutionRole-20241128T055392"
print("Role ARN:", role_arn) ## If local, Role ARN: arn:aws:iam::807711953667:role/voclabs
session = sagemaker.Session()
region = session.boto_region_name
bucket = session.default_bucket()
print("AWS Region: {}".format(region))
print("Default Bucket: {}".format(bucket))
print("Role Arn: {}".format(role_arn))

import wandb
## generate secrets.env. remember add it to .gitignore  
wandb.sagemaker_auth(path="scripts")  

[12/02/24 16:39:01] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=385428;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=557435;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[12/02/24 16:39:03] WARNING  Couldn't call 'get_role' to get Role ARN from role name voclabs to get ]8;id=439026;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=432725;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#5902\5902]8;;\
                             Role path.                                                                            

Role ARN: arn:aws:iam::061096721307:role/service-role/AmazonSageMaker-ExecutionRole-20241128T055392


                    INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=756274;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=497627;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

AWS Region: us-east-1
Default Bucket: sagemaker-us-east-1-061096721307
Role Arn: arn:aws:iam::061096721307:role/service-role/AmazonSageMaker-ExecutionRole-20241128T055392


In [8]:
from sagemaker.pytorch import PyTorch
from sagemaker.inputs import TrainingInput
from datetime import datetime
## Moving the 1.1GB data from one bucket to another takes 1 hours.
## This is roughly the same amount of time as uploading the data from a local machine to S3.
data_base_path = "s3://p3-dog-breed-classification/dogImages/"
train_data = TrainingInput(data_base_path+"train/", content_type="image/jpeg")
val_data = TrainingInput(data_base_path+"valid/", content_type="image/jpeg")
test_data = TrainingInput(data_base_path+"test/", content_type="image/jpeg")
output_path = "s3://p3-dog-breed-classification/jobs/"
hyperparameters = {
    'epochs': 20,  # Define how many epochs you want to train for
    'batch-size': 32,  ## ⚠️ probably small for smaller training dataset
    'opt-learning-rate': 5e-5,  ## optimizer lr. ⚠️ keep it small for pre-trained model
    'opt-weight-decay': 5e-5, ## optimizer weight decay
    'model-name': 'resnet50',  # Specify the ResNet model you want to use
}
# Define the PyTorch estimator
estimator = PyTorch(
    entry_point='train.py',  # Your training script that defines the ResNet50 model and training loop
    source_dir='scripts',  # Directory where your script and dependencies are stored
    role=role_arn,
    framework_version='1.13.1',  # Use the PyTorch version you need
    py_version='py39',
    instance_count=1,  # Adjust based on the number of instances you want to use
    # instance_type='ml.p3.2xlarge',  # Use GPU instances for deep learning
    instance_type='ml.g4dn.xlarge',
    output_path=output_path,
    hyperparameters=hyperparameters,
)

In [9]:
# Fit the estimator with the input channels (train, val)
estimator.fit(
    wait=True,  
    job_name=f"p3-dog-breeds-job-{datetime.now().strftime('%Y%m%d-%H%M%S')}",  
    inputs={
        "train": train_data, 
        "validation": val_data, 
        "test": test_data,
    },  
)
## baseline test accuracy 74%
## 48m 11.0s, 

[12/02/24 16:59:24] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=276277;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=66798;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#674\674]8;;\
                             instance_type, framework etc.                                                         

[12/02/24 16:59:25] INFO     Creating training-job with name: p3-dog-breeds-job-20241202-165923     ]8;id=921973;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=538361;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#1042\1042]8;;\

2024-12-02 22:59:30 Starting - Starting the training job...
2024-12-02 22:59:44 Starting - Preparing the instances for training...
2024-12-02 23:00:21 Downloading - Downloading input data......
2024-12-02 23:01:26 Downloading - Downloading the training image...............
2024-12-02 23:04:13 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.9/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.9/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


* 🟢⚠️ Issue solved:  
  * pytorch: [OSError: image file is truncated (150 bytes not processed)](https://discuss.pytorch.org/t/oserror-image-file-is-truncated-150-bytes-not-processed/64445)   
  * github: [OSError: image file is truncated (7 bytes not processed)](https://github.com/eriklindernoren/PyTorch-YOLOv3/issues/162)  
  * [OSError: image file is truncated 解决思路及方案](https://blog.csdn.net/qq_34097715/article/details/109646082)

```bash
File "/opt/conda/lib/python3.9/site-packages/torchvision/datasets/folder.py", line 268, in default_loader
    return pil_loader(path)
File "/opt/conda/lib/python3.9/site-packages/torchvision/datasets/folder.py", line 248, in pil_loader
    return img.convert("RGB")
File "/opt/conda/lib/python3.9/site-packages/PIL/Image.py", line 995, in convert
    self.load()
File "/opt/conda/lib/python3.9/site-packages/PIL/ImageFile.py", line 290, in load
    raise OSError(msg)
```